<a href="https://colab.research.google.com/github/aaron-v19/practice/blob/main/cnnpract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
#convert mnist image file into a Tensor of 4 dimensions(# of image, height,width, color code)
transform=transforms.ToTensor()

In [13]:
# Train data
train_data=datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

In [14]:
# Test data
test_data=datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

In [15]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [16]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [28]:
# Create a small batch size for images here 10
train_dataloader=DataLoader(train_data,batch_size=10,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=10,shuffle=True)

In [29]:
#define cnn model
conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1,padding=0)
conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1,padding=0)

In [31]:
#grab 1 mnist record
for i,(X_train,y_train) in enumerate(train_data):
  break

In [33]:
X_train.shape

torch.Size([1, 28, 28])

In [34]:
x=X_train.view(1,1,28,28)

In [35]:
#perform 1st convolution
x=F.relu(conv1(x))

In [36]:
# 1 single image, 6 filters, 26,26-> image dimensions
x.shape

torch.Size([1, 6, 26, 26])

In [38]:
#pass through pooling layer
x=F.max_pool2d(x,kernel_size=2,stride=2)

In [39]:
x.shape

torch.Size([1, 6, 13, 13])

In [40]:
x=F.relu(conv2(x))

In [41]:
x.shape

torch.Size([1, 16, 11, 11])

In [42]:
x=F.max_pool2d(x,kernel_size=2,stride=2)

In [43]:
x.shape

torch.Size([1, 16, 5, 5])

## Makeing model

In [45]:
#model class
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1,padding=0)
    self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1,padding=0)
    self.fc1=nn.Linear(in_features=5*5*16,out_features=120)
    self.fc2=nn.Linear(in_features=120,out_features=84)
    self.fc3=nn.Linear(in_features=84,out_features=10)

  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,kernel_size=2,stride=2)
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,kernel_size=2,stride=2)
    x=x.view(-1,5*5*16)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return F.log_softmax(x,dim=1)

In [46]:
#create manual seed
torch.manual_seed(41)
model=CNN()
model

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [47]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)